# Use Case
Choroby serca są główną przyczyną zgonów na świecie, powodując około 17,9 miliona osób rocznie, co stanowi około 32% wszystkich zgonów na świecie. Poniższy notebook umożliwia klasyfikowanie na podstawie danych w tabeli, czy dana osoba będzie narażona na choroby serca, czy nie. 
Dane pochodzą z [link](https://www.kaggle.com/sid321axn/heart-statlog-cleveland-hungary-final?select=heart_statlog_cleveland_hungary_final.csv). Zbiór ten łączy 11 cech, które umożliwiają ocenę, czy dana osoba będzie narażona na choroby serca.
Dane do pobrania z linku znajdują się w repozytorium jako `heart_disease.csv`

# Cel 
Przewidywania wartości w kolumnie `target` (narażony na chorobę serca) na podstawie pozostałych danych oraz znalezienie najlepszego algorytmu z listy dostępnych.

# Po co czytać? Jak można obejrzeć
[Youtube - film](https://youtu.be/eHDdKH4-Y1Y)

# Podstawy
1. Zaloguj się na [Azure Portal](https://portal.azure.com/#home)
2. Stwórz zasób Azure Machine Learning
3. Przejdź do Azure Machine Learning Studio
4. Dodaj zbiór - zakładka Datasets (tworzymy zbiór po przez wczytanie zbioru heart_disease.csv)
5. Stwórz Compute Instances oraz Compute Clusters - zakładka Compute 
- Compute Instance - potrzebny do obsługi Notebooka
- Compute Cluster - potrzebny do prowadzenia treningu
6. Następnie użyj Automated ML with UI lub Notebooks

# Automated ML with UI

W celu skorzystania należy w zakładzcie Automated ML:

1. Wybierz New Automated ML run
2. Wybieramy dodany zbiór danych
3. Konfigutacja 'biegu' po przez dodanie nazwy, wybranie stworzonego wcześniej klastra oraz wybranie kolumny, której wartości chcemy przewidywać.
4. Wybór typu zadania (klasyfikacja, regresja, przewidywanie wartości w czasie) - wybrano klasyfikacje
5. Po zakończonym procesie w jego szczególach w zakładce 'Models' możemy zobaczyć, że najlepszym algorytmem dla naszego zadania będzie `VotingEnsemble`, który osiągnął dokładność równą 93,445%

Ograniczono czas na uczenie do 2h.
Użycie graficznego interfejsu dla Automated ML umożliwi a trwyialny sposób na realizację tego zadania. Przeprowadzony eksperyment porównał poszczególne algorytmy i pokazał który będzie miał najlepszą dokładnośc dla wyznaczonego zadania.

# Użycie Notebooka

1. Przejdź do zakładki Notebooks

Wybierz jedną z dróg

Pierwsza:

2. Stwórz nowy Notebook i dołącz do niego Compute Instance.
3. Wczytaj plik `requirements.txt` do plików środowiska.
4. Skopiuj poszczególne komendy przedstawione poniżej i wywołaj w swoim Notebooku

Druga:

2. Wczytaj plik `requirements.txt` oraz `heart-notebook-pre.ipynb` do plików środowiska.
3. Otwórz plik Notebooka `heart-notebook-pre.ipynb`.
4. Wykonuj zapisane polecenia w kolejności. 

# Notebook

## Update do SDK version: 1.20.0 sprawił, że należy zainstalować potrzebne wersje narzędzi kodem poniżej.
Po zainstalowaniu potrzebnych wymagań zresetuj sesje (reset kernel)

In [1]:
pip install -r ./requirements.txt

Note: you may need to restart the kernel to use updated packages.


##  Uzyskaj obszar roboczy usługi Azure ML

In [1]:
# Wersja SDK powinna być 1.20.0 (stan na 24.01.20 rok - Microsoft lubi często coś zmieniać :/ )
import azureml.core

print("SDK version:", azureml.core.VERSION)

SDK version: 1.20.0


In [2]:
from azureml.core import Workspace, Dataset

# Uzyskaj obszar roboczy zdefiniowany w domyślnym pliku config.json
ws = Workspace.from_config()

## Załaduj dane z zestawów danych usługi Azure ML

Nazwa użytego zbioru to `heart-disease`.

In [3]:
# Wczytaj dane
aml_dataset = ws.datasets['heart-disease']

full_df = aml_dataset.to_pandas_dataframe()
full_df.head(5)

,age,sex,chest pain type,resting bp s,cholesterol,fasting blood sugar,resting ecg,max heart rate,exercise angina,oldpeak,ST slope,target
0,40,1,2,140,289,0,0,172,0,0.0,1,0
1,49,0,3,160,180,0,0,156,0,1.0,2,1
2,37,1,2,130,283,0,1,98,0,0.0,1,0
3,48,0,4,138,214,0,0,108,1,1.5,2,1
4,54,1,3,150,195,0,0,122,0,0.0,1,0


## Podział zbioru danych na dwie grupy: trenująca i testująca
Zbiór został podzielony tak, że 85% stanowi zbniór trenujący, a 15% testujący.

Zbiór zawiera 11 cech, które umożliwiają przewidywanie, więc żadna kolumna nie zostanie usunięta do dalszego procesu.

In [4]:
train_dataset, test_dataset = aml_dataset.random_split(0.85, seed=1)

train_dataset_df = train_dataset.to_pandas_dataframe()
test_dataset_df = test_dataset.to_pandas_dataframe()

print(train_dataset_df.describe())

               age          sex  chest pain type  resting bp s  cholesterol  \
count  1018.000000  1018.000000      1018.000000   1018.000000  1018.000000   
mean     53.613949     0.767191         3.228880    131.942043   209.581532   
std       9.184725     0.422830         0.939503     18.277700   101.383293   
min      28.000000     0.000000         1.000000      0.000000     0.000000   
25%      47.000000     1.000000         3.000000    120.000000   188.000000   
50%      54.000000     1.000000         4.000000    130.000000   228.000000   
75%      60.000000     1.000000         4.000000    140.000000   269.000000   
max      77.000000     1.000000         4.000000    200.000000   603.000000   

       fasting blood sugar  resting ecg  max heart rate  exercise angina  \
count          1018.000000  1018.000000     1018.000000      1018.000000   
mean              0.214145     0.695481      139.852652         0.389980   
std               0.410430     0.868774       25.045797     

## Połączenie do zdalnego AML Compute (Istniejący AML cluster)

Użyty klaster to `cpu-cluster`.

In [5]:
from azureml.core.compute import AmlCompute
from azureml.core.compute import ComputeTarget

# nazwa klastra
amlcompute_cluster_name = "cpu-clusters"

found = False
cts = ws.compute_targets

if amlcompute_cluster_name in cts and cts[amlcompute_cluster_name].type == 'AmlCompute':
     found = True
     print('Found existing training cluster.')
     # Get existing cluster
     # Method 1:
     aml_remote_compute = cts[amlcompute_cluster_name]
     # Method 2:
     # aml_remote_compute = ComputeTarget(ws, amlcompute_cluster_name)
    
if not found:
     print('Creating a new training cluster...')
     provisioning_config = AmlCompute.provisioning_configuration(vm_size = "STANDARD_D13_V2", # for GPU, use "STANDARD_NC12"
                                                                 #vm_priority = 'lowpriority', # optional
                                                                 max_nodes = 20)
     # Create the cluster.
     aml_remote_compute = ComputeTarget.create(ws, amlcompute_cluster_name, provisioning_config)
    
print('Checking cluster status...')
aml_remote_compute.wait_for_completion(show_output = True)

Found existing training cluster.
Checking cluster status...
Succeeded
AmlCompute wait for completion finished

Minimum number of nodes requested have been provisioned


## Zdefiniuj ustawienia eksperymentu AutoML (z AML Remote Compute)
Należy zdefiniować zadanie, metrykę, czas eksperymentu oraz nazwę kolumny.

W naszym przypadku jest to:

task = `'classification'`

primary_metric=`'accuracy'`

experiment_timeout_minutes=`15`    

label_column_name=`"target"`

In [6]:
import logging
import os

from azureml.train.automl import AutoMLConfig

project_folder = './heart-1'
os.makedirs(project_folder, exist_ok=True)

automl_config = AutoMLConfig(compute_target=aml_remote_compute,
                             task='classification',
                             primary_metric='accuracy',
                             experiment_timeout_minutes=15,                            
                             training_data=train_dataset,
                             label_column_name="target",
                             n_cross_validations=5,
                             # blacklist_models='XGBoostClassifier', 
                             # iteration_timeout_minutes=5,                                                    
                             enable_early_stopping=True,
                             featurization='auto',
                             debug_log='automated_ml_errors.log',
                             verbosity=logging.INFO,
                             path=project_folder
                             )

## Uruchom eksperyment (na komputerze zdalnym AML) z wieloma przebiegami podrzędnymi
W testowanych przypadkach ten krok zajmował ok 25-30 minut.

In [7]:
from azureml.core import Experiment
from datetime import datetime

now = datetime.now()
time_string = now.strftime("%m-%d-%Y-%H")
experiment_name = "classif-automl-remote-{0}".format(time_string)
print(experiment_name)

experiment = Experiment(workspace=ws, name=experiment_name)

import time
start_time = time.time()
            
run = experiment.submit(automl_config, show_output=True)

print('Manual run timing: --- %s seconds needed for running the whole Remote AutoML Experiment ---' % (time.time() - start_time))

classif-automl-remote-01-24-2021-23
Running on remote.
No run_configuration provided, running on cpu-clusters with default configuration
Running on remote compute: cpu-clusters
Parent Run ID: AutoML_6b204923-3c43-4242-9411-c78c90ed92a2

Current status: FeaturesGeneration. Generating features for the dataset.
Current status: DatasetFeaturization. Beginning to fit featurizers and featurize the dataset.
Current status: DatasetCrossValidationSplit. Generating individually featurized CV splits.
Current status: ModelSelection. Beginning model selection.

****************************************************************************************************
DATA GUARDRAILS: 

TYPE:         Class balancing detection
STATUS:       PASSED
DESCRIPTION:  Your inputs were analyzed, and all classes are balanced in your training data.
              Learn more about imbalanced data: https://aka.ms/AutomatedMLImbalancedData

*********************************************************************************

### Pokazanie szczegółów procesu oraz znalezenie najlepszego modelu

In [8]:
import time
import datetime as dt

run_details = run.get_details()

end_time_utc_str = run_details['endTimeUtc'].split(".")[0]
start_time_utc_str = run_details['startTimeUtc'].split(".")[0]
timestamp_end = time.mktime(datetime.strptime(end_time_utc_str, "%Y-%m-%dT%H:%M:%S").timetuple())
timestamp_start = time.mktime(datetime.strptime(start_time_utc_str, "%Y-%m-%dT%H:%M:%S").timetuple())

parent_run_time = timestamp_end - timestamp_start
print('Run Timing: --- %s seconds needed for running the whole Remote AutoML Experiment ---' % (parent_run_time))

#Best model
best_run, fitted_model = run.get_output()
print(best_run)
print(fitted_model)

Run Timing: --- 1230.0 seconds needed for running the whole Remote AutoML Experiment ---
Run(Experiment: classif-automl-remote-01-24-2021-23,
Id: AutoML_6b204923-3c43-4242-9411-c78c90ed92a2_16,
Type: azureml.scriptrun,
Status: Completed)
Pipeline(memory=None,
         steps=[('datatransformer',
                 DataTransformer(enable_dnn=None, enable_feature_sweeping=None,
                                 feature_sweeping_config=None,
                                 feature_sweeping_timeout=None,
                                 featurization_config=None, force_text_dnn=None,
                                 is_cross_validation=None,
                                 is_onnx_compatible=None, logger=None,
                                 observer=None, task=None, working_dir=None)),
                ('prefittedsoftvotingclassifier',...
                                                                                                    min_impurity_split=None,
                             

Najpelszy algorytm/model to `prefittedsoftvotingclassifier`

## Prognozowanie

### Wyodrębnij wartości X (kolumny funkcji) z testowego zestawu danych i przekonwertuj na tablicę NumPi w celu przewidywania

In [9]:
import pandas as pd

if 'target' in test_dataset_df.columns:
    y_test_df = test_dataset_df.pop('target')

x_test_df = test_dataset_df

### Dokonaj rzeczywistych prognoz

In [10]:
y_predictions = fitted_model.predict(x_test_df)

print('10 predictions: ')
print(y_predictions[:10])

10 predictions: 
[0 0 0 0 0 1 1 0 0 0]


In [11]:
y_predictions.shape

(172,)

### Oblicz dokładność za pomocą zestawu danych testowych

In [12]:
from sklearn.metrics import accuracy_score

print('Accuracy')
accuracy_score(y_test_df, y_predictions)

Accuracy


0.9418604651162791

Otrzymana dokładność tym sposobem wyniosła 94,19%, ale proces zamiast 2h został skrócony do 15 minut.